In [23]:
!pip install datasets


In [24]:
from datasets import load_dataset

# Replace 'emotion' with the name of the dataset you want to load
dataset = load_dataset("SKNahin/bengali-transliteration-data")

# Display the dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['bn', 'rm'],
        num_rows: 5006
    })
})


In [25]:
# Display the dataset's features and some sample data
print(dataset['train'].features)
print(dataset['train'][0])

# Display the first few rows of the dataset
for i in range(5):
    print(dataset['train'][i])

{'bn': Value(dtype='string', id=None), 'rm': Value(dtype='string', id=None)}
{'bn': 'স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???', 'rm': 'scroll kore 20/30 second er video pann nai???'}
{'bn': 'স্ক্রোল করে ২০/৩০ সেকেন্ড এর ভিডিও পান নাই???', 'rm': 'scroll kore 20/30 second er video pann nai???'}
{'bn': 'ও গুলা টরেন্ট সাইট এ পাবেন', 'rm': 'o gula Torrent site e paben'}
{'bn': 'ভক্কর চক্কর পোস্ট একটা করলেই এপ্রুভড.… নিশ্চই  ঘাবলা আছে', 'rm': 'vokkor chokkor post akta korlei approved…. nishchoi ghabla ache'}
{'bn': 'আমি টেস্ট করেই কোড দিছি…', 'rm': 'ami test koreii code disi…'}
{'bn': 'এতো কষ্টের কি আছে সাকিবওয়াপ.টক,সাকিবওয়াপ.মল&এআইওভিডিওড্ল.মল থেকে সহজেই ডাউনলোড করা যায়', 'rm': 'eto koster ki ache shakibwap.tk,shakibwap.ml&aiovideodl.ml theke shohojei downlod kora jay'}


In [26]:
# Split the dataset
split_dataset = dataset['train'].train_test_split(test_size=0.1)

# Extract train and test sets
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']


In [27]:
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 4505
Test dataset size: 501


In [28]:
from transformers import T5Tokenizer

In [29]:
# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Tokenize data
def tokenize_data(example):
    # Tokenize the Banglish (input) text
    inputs = tokenizer(example['rm'], truncation=True, padding='max_length', max_length=128)

    # Tokenize the Bangla (target) text
    targets = tokenizer(example['bn'], truncation=True, padding='max_length', max_length=128)

    # Add decoder_input_ids explicitly
    inputs['labels'] = targets['input_ids']  # The 'labels' will be the target input_ids for the decoder

    return inputs

    # return tokenizer(example['rm'], truncation=True, padding='max_length', max_length=128)

train_dataset = train_dataset.map(tokenize_data, batched=True)
test_dataset = test_dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/4505 [00:00<?, ? examples/s]

Map:   0%|          | 0/501 [00:00<?, ? examples/s]

In [30]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [31]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [32]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=500,
    save_total_limit=2,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [33]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-33-d61bf35e79ab>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [34]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.509600,0.028626
2,0.030800,0.020503
3,0.025900,0.019869


TrainOutput(global_step=1692, training_loss=0.1702677076308158, metrics={'train_runtime': 302.2237, 'train_samples_per_second': 44.719, 'train_steps_per_second': 5.599, 'total_flos': 457286111723520.0, 'train_loss': 0.1702677076308158, 'epoch': 3.0})

In [35]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print the evaluation results
print(eval_results)

{'eval_loss': 0.01986859366297722, 'eval_runtime': 2.8526, 'eval_samples_per_second': 175.631, 'eval_steps_per_second': 22.085, 'epoch': 3.0}
